Importing Necessary Libraries

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray
import matplotlib.pyplot as plt



Declaring the Test Image Set and Outputset and Importing Saved Models

In [ ]:
#input(test) and output directories
test_dir = 'C:/Users/GOKAY/Desktop/imagenette4/test/'
output_ae_dir = 'C:/Users/GOKAY/Desktop/compressed_images/'

#loading models
ae1 = load_model('C:/Users/GOKAY/Desktop/tez_kod/ae1.keras')
ae2 = load_model('C:/Users/GOKAY/Desktop/tez_kod/ae2.keras')
ae3 = load_model('C:/Users/GOKAY/Desktop/tez_kod/ae3.keras')


Creating a Function to Evaluate Autoencoders and Saving Pictures

In [ ]:
def evaluate_ae(ae):
    psnr_values = []
    ssim_values = []
    i = 0
    #the loops takes one image at a time and predicts it by the autoencoder, so that evaluation can be done on each image seperately
    for img in os.listdir(test_dir):
        if img.endswith(".JPEG"):
            #Importing an image from test file and turning into an array for evaluation
            img_path = os.path.join(test_dir, img)
            img = load_img(img_path)
            img_array = img_to_array(img)
            #expanding dimension of the image for autoencoder to predict
            img_array_expanded = np.expand_dims(img_array, axis=0)

            #predicting the image and getting it as output
            output = ae.predict(img_array_expanded)
            output_image = output[0]

            #saving and visualising 20 images of predictions
            if i < 20:
                plt.figure(figsize=(1, 1))
                plt.imshow(image.array_to_img(output_image))
                plt.gray()
                plt.axis('off')
                filename = f"{output_ae_dir}image{i}.png"    
                i+=1
                plt.savefig(filename, bbox_inches='tight', pad_inches=0)
            
            #psnr value is received and added to the list
            current_psnr = psnr(img_array, output_image, data_range=255)
            psnr_values.append(current_psnr)

            #images are turned into grayscale for ssim evaluation
            gray_original = rgb2gray(img_array)
            gray_output = rgb2gray(output_image)
            
            #ssim value is received and added to the list
            current_ssim = ssim(gray_original, gray_output, data_range=255)
            ssim_values.append(current_ssim)

    #mean values of metrics are calculated as final result and returned
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_psnr, mean_ssim

In [ ]:
print("Result for first autoencoder:",evaluate_ae(ae1))
print("Result for second autoencoder:",evaluate_ae(ae2))
print("Result for third autoencoder:",evaluate_ae(ae3))

Preparing The Directories of Images of Conventional Methods

In [ ]:
directory = 'C:/Users/GOKAY/Desktop/compressed_images/conv/'
sub_directories = [
    'jpeg 25/',
    'jpeg 50/',
    'jpeg 75/',
    'WEBP 25/',
    'WEBP 50/',
    'WEBP 75/',
    'tiff/'
]      

Creating a Function to Evaluate Conventional Methods

In [ ]:
def evaluate_conv(dir):
    psnr_values = []
    ssim_values = []
    #same as in evaluation of autoencoders, evaluation is done one at a time for each image
    for img in os.listdir(test_dir):
        if img.endswith(".JPEG"):
            #importing an image and turning it into an array for evaluation
            org_img_path = os.path.join(test_dir, img)
            org_img = load_img(org_img_path)
            org_img_array = img_to_array(org_img)

            #finding the same image in the compressed image directories of conventional methods
            if dir.lower().startswith("jpeg"):
                comp_img_path = f'{directory}{dir}{img}'
            elif dir.lower().startswith("tiff"):
                comp_img_path = f'{directory}{dir}{img[:-5]}.tiff'
            else:
                comp_img_path = f'{directory}{dir}{img[:-5]}.webp'
            
            #receiving the compressed image and turning it into an array
            rec_img = load_img(comp_img_path)
            rec_img_array = img_to_array(rec_img)
            
            #psnr value is received and added to the list
            current_psnr = psnr(org_img_array, rec_img_array, data_range=255)
            psnr_values.append(current_psnr)

            #image arrays are turned into gray scale for ssim evaluation
            gray_original = rgb2gray(org_img_array)
            gray_compressed = rgb2gray(rec_img_array)

            #ssim values is received and added to the list
            current_ssim = ssim(gray_original, gray_compressed, data_range=255)
            ssim_values.append(current_ssim)

    #mean values of metrics are calculated as final result and returned 
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_psnr, mean_ssim

In [ ]:
print("Result for jpeg 25:",evaluate_conv(sub_directories[0]))
print("Result for jpeg 50:",evaluate_conv(sub_directories[1]))
print("Result for jpeg 75:",evaluate_conv(sub_directories[2]))
print("Result for webp 25:",evaluate_conv(sub_directories[3]))
print("Result for webp 50:",evaluate_conv(sub_directories[4]))
print("Result for webp 75:",evaluate_conv(sub_directories[5]))
print("Result for tiff:",evaluate_conv(sub_directories[6]))